In [5]:
# @title Installs and imports (run me first!)
#import nltk
#nltk.download('punkt')

import flax
from graphviz import Digraph
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
import numpy as onp

from IPython.display import HTML
from pprint import pprint
import logging
from pyvirtualdisplay import Display
logging.getLogger("pyvirtualdisplay").setLevel(logging.ERROR)


import os


In [6]:
import numpy as np


In [7]:
import gym
#gym.make('CartPole-v1')

In [8]:
env = gym.make("gym_fake:fake-v0")
#env=gym.make('CartPole-v1')

2022-03-03 02:32:14.704126: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNor

In [9]:
from dopamine.jax import networks
from dopamine.jax.agents.dqn import dqn_agent
import gin
import numpy as np

ImportError: cannot import name 'DopamineKit' from partially initialized module 'dopaminekit' (most likely due to a circular import) (/home/alberto/.local/lib/python3.8/site-packages/dopaminekit/__init__.py)

In [ ]:
from flax import linen as nn

from dopamine.discrete_domains import atari_lib
class FakeDQNNetwork(nn.Module):
  """Jax DQN network for Cartpole."""
  num_actions: int

  @nn.compact
  def __call__(self, x):
    initializer = nn.initializers.xavier_uniform()
    x = x.astype(jnp.float32)
    print(1)
    x = x.reshape((-1))  # flatten
    #x -= np.array(np.ones(768)*(-10))
    #print(2)
    #x /= np.array(np.ones(768)*(-10)) - np.array(np.ones(768)*(-10))
    #x = 2.0 * x - 1.0  # Rescale in range [-1, 1].
    x = nn.Dense(features=512, kernel_init=initializer)(x)
    x = nn.relu(x)
    x = nn.Dense(features=512, kernel_init=initializer)(x)
    x = nn.relu(x)
    print(3)
    q_values = nn.Dense(features=self.num_actions, kernel_init=initializer)(x)
    print(4)
    return atari_lib.DQNNetworkType(q_values)

In [ ]:
Fake_config = """
JaxDQNAgent.gamma = 0.99
JaxDQNAgent.update_horizon = 1
JaxDQNAgent.min_replay_history = 500
JaxDQNAgent.update_period = 4
JaxDQNAgent.target_update_period = 100
JaxDQNAgent.epsilon_fn = @dqn_agent.identity_epsilon

create_optimizer.name = 'adam'
create_optimizer.learning_rate = 0.001
create_optimizer.eps = 3.125e-4

OutOfGraphReplayBuffer.replay_capacity = 50000
OutOfGraphReplayBuffer.batch_size = 128
"""
gin.parse_config(Fake_config, skip_unknown=False)


In [ ]:
dqn_agent = dqn_agent.JaxDQNAgent(num_actions=8,
                                  observation_shape=(1538, 1),
                                  observation_dtype=jnp.float64,
                                  stack_size=1,
                                  network=FakeDQNNetwork)

In [ ]:
max_steps_per_episode = 4  # @param {type:'slider', min:10, max:1000}
training_steps = 4  # @param {type:'slider', min:10, max:5000}
num_iterations = 4 #@param {type:"slider", min:10, max:200, step:1}

# First remove eval mode!
dqn_agent.eval_mode = False
average_returns = []
# Each iteration will consist of a number of episodes.
for iteration in range(num_iterations):
  step_count = 0
  num_episodes = 0
  sum_returns = 0.
  # This while loop will continue running episodes until we've done enough
  # training steps.
  while step_count < training_steps:
    episode_length = 0
    episode_rewards = 0.
    s = env.reset()
    a = int(dqn_agent.begin_episode(s))
    
    is_terminal = False
    # Run the episode until termination.
    while True:
      #r=env.step(a)[1]
      #s=env.step(a)[0]
      #done=env.step(a)[2]
      b=env.step(a)
      s, r, done, _ = b
      episode_rewards += r
      episode_length += 1
      if done or episode_length == max_steps_per_episode:
        # Stop the loop if the episode or ended or we've reached the max steps.
        break
      else:
        a = int(dqn_agent.step(r, s))
    dqn_agent.end_episode(r)
    step_count += episode_length
    sum_returns += episode_rewards
    num_episodes += 1
  average_return = sum_returns / num_episodes if num_episodes > 0 else 0.
  print(f'Iteration {iteration}: average return = {average_return:.4f}')
  average_returns.append(average_return)

In [ ]:
def learned_policy(s):
  return dqn_agent.step(0., s)  # We pass in a dummy reward

# We set our agent in `eval_mode` to avoid it from continuing to train while
# interacting with the environment.
dqn_agent.eval_mode = True
animate_agent(learned_policy, cartpole_env, num_frames=100)

In [ ]:
dqn_agent.bundle_and_checkpoint('/content/gdrive/MyDrive/google/diaspora/',1)